# Step1 Getting familiar with the muon objects and nanoaod
In this exercise we will get familiar with the muon objects in an [nanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) file. We will mostly use the following file, containing simulated Drell–Yan dimuon events generated at NLO.<br>
* DYJetsToMuMu_M-50_TuneCP5_13TeV-madgraphMLM-pythia8.root

Further information and a list of those variables stored in nanoAOD can be found [here](https://cms-nanoaod-integration.web.cern.ch/integration/master-106X/mc102X_doc.html).<br>
In nanoAOD, data are sored as arrays. 
[Uproot](https://github.com/scikit-hep/uproot#exploring-a-file) will be used to explore the nanoAOD file.


Not importing? Don't forget to select the correct kernel

In [ ]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
from math import pi

First, lets take a look of the variables stored in NanoAOD. Uproot.open( ) return a ROOTDirectory similar to a python dictionary. Function key( ) prints all the dicinory keywords. The b' ahead of the names is due to ROOT object names having no encoding.

In [ ]:
events=uproot.open('/cms/store/user/hats/Muon/DYJetsToMuMu_M-50_TuneCP5_13TeV-madgraphMLM-pythia8.root')['Events']
#events=uproot.open('~/Downloads/DYJetsToMuMu_M-50_TuneCP5_13TeV-madgraphMLM-pythia8.root')['Events']

In [ ]:
events.keys()
#events.values()
#events.show()

**Question 1**: Which collections contain muons or muon candidates? (At any level: generated, reconstructed, trigger, etc.)

# Step2 Print and plot some muon objects 
We can print different objects such as transeverse momentum($p_T$ pt), pseudorapidity(eta $\eta$) and azimuthal angle(phi $\phi$) and apply cut on $p_T$, $\eta$ and $\phi$ using uproot.<br>
First lets print pt eta phi of reconstructed muon 

In [ ]:
muon_pt,muon_eta,muon_phi,nmuon,ngenpart=events.arrays(["Muon_pt", "Muon_eta", "Muon_phi", "nMuon", "nGenPart"],outputtype=tuple)

In [ ]:
print('muon_pt=', muon_pt, '\n',
      'muon_eta=', muon_eta, '\n',
      'muon_phi=', muon_phi)

Lets plot them using matplotlib

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_pt.flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='blue', label='$p_T$ distribution of all Muons')
plt.title('$p_T$ distribution of PAT Muon objects')
plt.xlabel(r'Muon $p_T$ (GeV)')
plt.ylabel('Events')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_eta.flatten(), bins=60, range=[-3, 3], histtype='step',linewidth=2, color='red', label='muon_$\eta$')
plt.title('$\eta$ distribution of PAT Muon objects')
plt.xlabel(r'Muon $\eta$')
plt.ylabel('Events')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_phi.flatten(), bins=70, range=[-3.5, 3.5], histtype='step',linewidth=2, color='green', label='muon_$\phi$')
plt.title('$\phi$ distribution of PAT Muon objects')
plt.xlabel(r'Muon $\phi$')
plt.ylabel('Events')

Then lets apply criteria such as pt>20 GeV and |eta|<2.4.

In [ ]:
masking=(muon_pt>20)&(abs(muon_eta)<2.4)
masking

As you can see above, by performing masking, a jagged array of bool values is created.
Then using the sytax array[masking], we keep only the varible that is true in the masking array

In [ ]:
print('muon_pt=', muon_pt[masking], '\n',
      'muon_eta=', muon_eta[masking], '\n',
      'muon_phi=', muon_phi[masking])

Then lets plot pt of reconstructed muon and reconstructed muon with pt>20 GeV and |eta|<2.4, note that we can use flatten() to reduce the jagged 2 dimension array to 1 dimension.

In [ ]:
muon_pt_masked=muon_pt[masking]
muon_pt_flat=muon_pt.flatten()
muon_pt_masked_flat=muon_pt_masked.flatten()

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_pt_flat, bins=150, range=[0,150], histtype='step',linewidth=2, color='red', label='muon_$p_T$')
plt.hist(muon_pt_masked_flat, bins=150, range=[0,150], histtype='step',linewidth=2, linestyle=':', color='blue', label='muon_$p_T$ with criteria')
plt.legend()
plt.xlabel(r'Muon $p_T$ (GeV)')
plt.ylabel('Events')

**Question 2**: Using the syntax above, plot the main kinematic observables for the following objects and selections (only the first few events):
* all final-state muons in the GenPart collection
* all final-state muons in the GenPart collection with pT > 20 GeV and |η| < 2.4
* all final-state muons in the GenPart collection with pT > 20 GeV and |η| < 2.4, and not coming from a hadron or τ decay

**Question 3**: Compare the numbers of reconstructed and generated muons and their pT spectra at different selection stages, by overlaying the distributions in the same frame.<br>
Since you are running on a Drell–Yan sample, you would expect two muons per event. Explain why there are sometimes more than two muons in an event.<br>
Look at the full distributions, without η and pT cuts. Can you explain the differences?<br>

Hint (GenPart_statusFlags: gen status flags stored bitwise. Bits are: 0 : isPrompt, 1 : isDecayedLeptonHadron, 2 : isTauDecayProduct, 3 : isPromptTauDecayProduct, 4 : isDirectTauDecayProduct, 5 : isDirectPromptTauDecayProduct, 6 : isDirectHadronDecayProduct, 7 : isHardProcess, 8 : fromHardProcess, 9 : isHardProcessTauDecayProduct, 10 : isDirectHardProcessTauDecayProduct, 11 : fromHardProcessBeforeFSR, 12 : isFirstCopy, 13 : isLastCopy, 14 : isLastCopyBeforeFSR,

GenPart_status	Particle status. 1=stable)

In [ ]:
genpart_status, genpart_statusflags, genpart_pdgid, genpart_pt, genpart_eta, genpart_phi=events.arrays(["GenPart_status", "GenPart_statusFlags", "GenPart_pdgId", "GenPart_pt", "GenPart_eta", "GenPart_phi"], outputtype=tuple)
masking_one=(abs(genpart_pdgid)==13)& (genpart_status==1)
masking_two=(abs(genpart_pdgid)==13)& (genpart_status==1)&(genpart_pt>20)&(abs(genpart_eta)<2.4)
masking_three=(abs(genpart_pdgid)==13)& (genpart_status==1)&(genpart_pt>20)&(abs(genpart_eta)<2.4)&((genpart_statusflags%2)==1)
plt.figure(figsize=(9,6))
plt.hist(genpart_pt[masking_one].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='red', label='All final state Muons')
plt.hist(genpart_pt[masking_two].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='blue', label='Final state Muons with $p_T$ > 20GeV and $|\eta|$<2.4')
plt.hist(genpart_pt[masking_three].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='brown', linestyle=('dashed'), label='Final state Muons with $p_T$ > 20GeV and $|\eta|$<2.4\nand not coming from hadron or $\\tau$ decay')
plt.legend()
plt.title('$p_T$ distribution of the Generated Muons')
plt.xlabel(r'generated Muon $p_T$ (GeV)')
plt.ylabel('Events')
plt. yscale('log')

Lets also compare the number of muons in each event between reconstructed one and generated one

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(nmuon.flatten(), bins=10, range=[0,10], histtype='step',linewidth=2, color='red', label='number of PAT muons')
plt.hist(masking_one.sum().flatten(), bins=10, range=[0,10], histtype='step',linewidth=2, color='blue', label='number of GEN muons')
plt.legend()
plt.xlabel('number of muon')
plt.ylabel('Events')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_pt.flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='blue', label='recon_muon_$p_T$')
plt.hist(genpart_pt[masking_one].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='red', label='gen_muon_$p_T$')
plt.legend()
plt.xlabel(r'Muon $p_T$ (GeV)')
plt.ylabel('Events')

# Step 3 Gen matching
When working with simulated data, it is sometime useful to pair each reconstructed muon to its corresponding generated particle, e.g. for efficiency or resolution studies. In this part of the exercise we will learn possible ways to do this.<br>

Gen matching information for muon is already stored in Muon_genPartIdx, which is matched on status=1 muon using particle flow algorithm. 
Lets print the Moun_genPartIdx. Note that -1 means cannot matched to generated particles.

In [ ]:
muon_genpartidx=events.array("Muon_genPartIdx")
muon_genpartidx

We can also do a matching ourselves using deltaR method, in deltaR method, we need to make sure the delta phi is between pi and -pi. Below defines a deltaR function
&Delta;R = (&Delta;&eta;<sup>2</sup> + &Delta;&phi;<sup>2</sup>)<sup>1/2</sup> <br>
Note that &Delta;&phi; is between &pi; to -&pi;.

In [ ]:
def deltaR(genetas, eta, genphis, phi):
    return np.sqrt((genetas-eta)**2+((genphis-phi+pi)%(2*pi)-pi)**2)

In [ ]:
muon_genpartidx_deltar=[]

for genetas, genphis, etas, phis in zip(genpart_eta, genpart_phi, muon_eta, muon_phi):
  idx=[]
  for eta, phi in zip(etas, phis):
    deltar=deltaR(genetas, eta, genphis, phi)
    if deltar.min()<0.3:
        idx.append(deltar.argmin())
    else:
        idx.append(-1)
  muon_genpartidx_deltar.append(idx)

#print(muon_genpartidx_deltar)
#for geneta, genphi, in zip

**Question 5**: Are there reconstructed muons that do not match any generated muons? Why? 

**Question 6**: Are there cases in which the &Delta;R matching fails to find the correct generated particle? Why? 

Lets take a look at those reconstracted muons that do not match any generated muons, and plot their pt and eta distribution

In [ ]:
masking_nogenmatch=(muon_genpartidx==-1)
masking_genmatched=(muon_genpartidx!=-1)

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_pt[masking_nogenmatch].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='blue', label='unmatched PAT muon $p_T$')
plt.hist(muon_pt[masking_genmatched].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='red', label='matched PAT muon $p_T$')
plt.legend()
plt.xlabel(r'PAT Muon $p_T$ (GeV)')
plt.ylabel('Events')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(muon_eta[masking_nogenmatch].flatten(), bins=60, range=[-3,3], histtype='step',linewidth=2, color='blue', label='unmatched PAT muon $\eta$')
plt.hist(muon_eta[masking_genmatched].flatten(), bins=60, range=[-3,3], histtype='step',linewidth=2, color='red', label='matched PAT muon $\eta$')
plt.legend()
plt.xlabel(r'PAT Muon $\eta$')
plt.ylabel('Events')

# Step 4: Trigger matching

A physics analysis often requires a matching between trigger and offline objects for a proper assessment of trigger efficiencies and related systematics. In order to reduce size per event. NanoAODThe no longer store matching information between muon objects and trigger objects. 
TrigObj_id in nanoAOD stored the ID of the HLT object. Using previous recipe, we can perform a simple $\Delta R$ matching between HLT and reconstructed objects.

In [ ]:
hlt_pt, hlt_eta, hlt_phi, hlt_id= events.arrays(["TrigObj_pt", "TrigObj_eta", "TrigObj_phi", "TrigObj_id"], outputtype=tuple)

In [ ]:
hltmuon_mask=abs(hlt_id)==13

In [ ]:
hltmuon_genpartidx_deltar=[]

for genetas, genphis, etas, phis in zip(genpart_eta, genpart_phi, hlt_eta[hltmuon_mask], hlt_phi[hltmuon_mask]):
  idx=[]
  for eta, phi in zip(etas, phis):
    deltar=deltaR(genetas, eta, genphis, phi)
    if deltar.min()<0.1:
        idx.append(deltar.argmin())
    else:
        idx.append(-1)
  hltmuon_genpartidx_deltar.append(idx)


In [ ]:
hltmuon_genpartidx_deltar=ak.fromiter(hltmuon_genpartidx_deltar)
hltmasking_nogenmatch=(hltmuon_genpartidx_deltar==-1)
hltmasking_genmatched=(hltmuon_genpartidx_deltar!=-1)

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(hlt_pt[hltmuon_mask].flatten(), bins=150, range=[0,150], histtype='step', linewidth=2, color='green', label='HLT muon $p_T$')
plt.hist(hlt_pt[hltmuon_mask][hltmasking_nogenmatch].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='blue', label='unmatched HLT muon $p_T$')
plt.hist(hlt_pt[hltmuon_mask][hltmasking_genmatched].flatten(), bins=150, range=[0,150], histtype='step',linewidth=2, color='red', linestyle=(':'), label='matched HLT muon $p_T$')

plt.legend()
plt.xlabel(r'Muon $p_T$ (GeV)')
plt.ylabel('Events')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(hlt_eta[hltmuon_mask][hltmasking_nogenmatch].flatten(), bins=60, range=[-3,3], histtype='step',linewidth=2, color='blue', label='unmatched HLT muon $\eta$')
plt.hist(hlt_eta[hltmuon_mask][hltmasking_genmatched].flatten(), bins=60, range=[-3,3], histtype='step',linewidth=2, color='red', label='matched HLT muon $\eta$')
plt.hist(hlt_eta[hltmuon_mask].flatten(), bins=60, range=[-3,3], histtype='step',linewidth=2, color='green', label='HLT muon $\eta$')
plt.legend()
plt.xlabel(r'Muon $\eta$')
plt.ylabel('Events')